In [1]:
import random
from typing import List, Dict, Any, Optional, Tuple
from pydantic import BaseModel, Field
from abstractions.goap.entity import Statement
from abstractions.goap.spatial import GridMap, Node, GameEntity, ActionResult,ActionsPayload, SummarizedActionPayload,SummarizedEgoActionPayload
from abstractions.goap.interactions import MoveStep, PickupAction, DropAction, OpenAction, CloseAction, UnlockAction, LockAction
from abstractions.goap.game.main import generate_dungeon
from abstractions.goap.agents import  AgentGoal, FakeLLM, LLMAgent, CharacterAgent, RunMetadata, system_prompt, action_generation_prompt,OutlinesActionPayload, OutlinesEgoActionPayload
import outlines
from outlines import models, generate



pygame-ce 2.4.1 (SDL 2.28.5, Python 3.12.1)


/Users/tommasofurlanello/.pyenv/versions/3.12.1/envs/abstractions/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = "/Users/tommasofurlanello/Documents/Dev/models/NousResearch/Hermes-2-Pro-Mistral-7B-GGUF/Hermes-2-Pro-Mistral-7B.Q8_0.gguf"
# model_path = "mistralai/Mistral-7B-Instruct-v0.2"
# model = models.llamacpp(model_path, model_kwargs={"seed": 1337, "n_ctx": 8000,"n_gpu_layers":-1})


In [3]:
def source_node_at_00(source: Node, target: Optional[Node] = None) -> bool:
    """ Check if the source node is at position (0, 0)"""
    print("Checking if source node at 0, 0", source.position)
    if source.node.position.value == (0, 0):
        print("Source node at 0, 0", source.node.position)
        return True
    return False

# Create the grid map and generate the dungeon
grid_map = GridMap(width=10, height=10)
grid_map.register_actions([MoveStep, PickupAction, DropAction, OpenAction, CloseAction, UnlockAction, LockAction])
room_width, room_height = 6, 6
character, door, key, treasure = generate_dungeon(grid_map, room_width, room_height)

# Generate the entity type map
grid_map.generate_entity_type_map()

# Create the LLMAgent
agent = LLMAgent(grid_map, character.id, model_path, use_egocentric=False,memory_length=5)

# Create the CharacterAgent with the LLMAgent
character_agent = CharacterAgent(grid_map, agent)

# Set the agent's metadata
metadata = RunMetadata(character_id=character.id, run_number=1)
character_agent.set_metadata(metadata)

# Set the agent's goals
goal_statement = Statement(callables=[source_node_at_00])  # Example goal: reach position (0, 0)
goal = AgentGoal(statement=goal_statement, priority=1)
character_agent.set_goals([goal])

# Get the initial observation
initial_observation = character_agent.get_current_observation()



llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from /Users/tommasofurlanello/Documents/Dev/models/NousResearch/Hermes-2-Pro-Mistral-7B-GGUF/Hermes-2-Pro-Mistral-7B.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = jeffq
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32           

Updating block attributes... for door


AVX = 0 | AVX_VNNI = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 0 | NEON = 1 | ARM_FMA = 1 | F16C = 0 | FP16_VA = 1 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 0 | SSSE3 = 0 | VSX = 0 | MATMUL_INT8 = 0 | 
Model metadata: {'general.quantization_version': '2', 'tokenizer.chat_template': "{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}", 'tokenizer.ggml.add_eos_token': 'false', 'tokenizer.ggml.add_bos_token': 'true', 'tokenizer.ggml.eos_token_id': '32000', 'tokenizer.ggml.bos_token_id': '1', 'tokenizer.ggml.model': 'llama', 'llama.attention.head_count_kv': '8', 'llama.context_length': '32768', 'llama.attention.head_count': '32', 'llama.rope.freq_base': '10000.000000', 'llama.rope.dimension_count': '128', 'general.file_type': '7', 'llama.feed_forward_length': '14336', 'llama.embedding_length': '4096', 'llama.block_count':

In [4]:
character_agent.run(initial_observation)


--- Step: 0 ---



llama_print_timings:        load time =     880.62 ms
llama_print_timings:      sample time =       6.73 ms /    79 runs   (    0.09 ms per token, 11738.48 tokens per second)
llama_print_timings: prompt eval time =    4276.77 ms /  2912 tokens (    1.47 ms per token,   680.89 tokens per second)
llama_print_timings:        eval time =    2496.73 ms /    78 runs   (   32.01 ms per token,    31.24 tokens per second)
llama_print_timings:       total time =    6915.02 ms /  2990 tokens


Generated Action: action_name='MoveStep' target_entity_type='Floor' target_entity_position=[5, 1] explanation_of_my_behavior='I choose to move towards the character in order to interact with it and possibly progress towards my goal.'
Generated Action: action_name='MoveStep' target_entity_type='Floor' target_entity_position=[5, 1] explanation_of_my_behavior='I choose to move towards the character in order to interact with it and possibly progress towards my goal.'
Payload detected action_name='MoveStep' target_entity_type='Floor' target_entity_position=[5, 1] explanation_of_my_behavior='I choose to move towards the character in order to interact with it and possibly progress towards my goal.'
Converting to summarized from outlines
Converting summarized payload action_name='MoveStep' source_entity_type='Character' source_entity_position=(5, 1) source_entity_id=None source_entity_name=None source_entity_attributes={} target_entity_type='Floor' target_entity_position=(5, 1) target_entity_i


llama_print_timings:        load time =     880.62 ms
llama_print_timings:      sample time =       6.65 ms /    77 runs   (    0.09 ms per token, 11578.95 tokens per second)
llama_print_timings: prompt eval time =    4237.67 ms /  2912 tokens (    1.46 ms per token,   687.17 tokens per second)
llama_print_timings:        eval time =    2463.10 ms /    76 runs   (   32.41 ms per token,    30.86 tokens per second)
llama_print_timings:       total time =    6840.57 ms /  2988 tokens


Generated Action: action_name='MoveStep' target_entity_type='Floor' target_entity_position=[5, 1] explanation_of_my_behavior='I am moving towards the character to potentially interact with it or gather more information about its presence.'
Generated Action: action_name='MoveStep' target_entity_type='Floor' target_entity_position=[5, 1] explanation_of_my_behavior='I am moving towards the character to potentially interact with it or gather more information about its presence.'
Payload detected action_name='MoveStep' target_entity_type='Floor' target_entity_position=[5, 1] explanation_of_my_behavior='I am moving towards the character to potentially interact with it or gather more information about its presence.'
Converting to summarized from outlines
Converting summarized payload action_name='MoveStep' source_entity_type='Character' source_entity_position=(5, 1) source_entity_id=None source_entity_name=None source_entity_attributes={} target_entity_type='Floor' target_entity_position=(5, 

...


llama_print_timings:        load time =     880.62 ms
llama_print_timings:      sample time =       8.43 ms /    94 runs   (    0.09 ms per token, 11146.69 tokens per second)
llama_print_timings: prompt eval time =    5205.79 ms /  2907 tokens (    1.79 ms per token,   558.42 tokens per second)
llama_print_timings:        eval time =    3171.98 ms /    93 runs   (   34.11 ms per token,    29.32 tokens per second)
llama_print_timings:       total time =    8559.87 ms /  3000 tokens


Generated Action: action_name='MoveStep' target_entity_type='Floor' target_entity_position=[5, 0] explanation_of_my_behavior="I am moving towards the location of the 'Statement' goal, which is currently at position (5, 0). This action allows me to make progress towards my objective."
Generated Action: action_name='MoveStep' target_entity_type='Floor' target_entity_position=[5, 0] explanation_of_my_behavior="I am moving towards the location of the 'Statement' goal, which is currently at position (5, 0). This action allows me to make progress towards my objective."
Payload detected action_name='MoveStep' target_entity_type='Floor' target_entity_position=[5, 0] explanation_of_my_behavior="I am moving towards the location of the 'Statement' goal, which is currently at position (5, 0). This action allows me to make progress towards my objective."
Converting to summarized from outlines
Converting summarized payload action_name='MoveStep' source_entity_type='Character' source_entity_position=


llama_print_timings:        load time =     880.62 ms
llama_print_timings:      sample time =       8.29 ms /    76 runs   (    0.11 ms per token,  9163.25 tokens per second)
llama_print_timings: prompt eval time =    5184.62 ms /  2907 tokens (    1.78 ms per token,   560.70 tokens per second)
llama_print_timings:        eval time =    2577.74 ms /    75 runs   (   34.37 ms per token,    29.10 tokens per second)
llama_print_timings:       total time =    7937.32 ms /  2982 tokens


Generated Action: action_name='MoveStep' target_entity_type='Floor' target_entity_position=[6, 0] explanation_of_my_behavior="I am choosing to move towards the character's position in order to potentially interact with it or gather more information about the environment."
Generated Action: action_name='MoveStep' target_entity_type='Floor' target_entity_position=[6, 0] explanation_of_my_behavior="I am choosing to move towards the character's position in order to potentially interact with it or gather more information about the environment."
Payload detected action_name='MoveStep' target_entity_type='Floor' target_entity_position=[6, 0] explanation_of_my_behavior="I am choosing to move towards the character's position in order to potentially interact with it or gather more information about the environment."
Converting to summarized from outlines
Converting summarized payload action_name='MoveStep' source_entity_type='Character' source_entity_position=(5, 0) source_entity_id=None source_

KeyboardInterrupt: 

In [ ]:
initial_observation

"Entity Types: ['Floor'], Attributes: [('BlocksLight', False), ('BlocksMovement', False), ('Material', '')], Positions: (0:2, 0:3), (8:10, 0:2), (2, 0:2), (3, 0:2), (4, 0:2), (6, 0:2), (7, 0:2), (5, 0), (9, 2)\nEntity Types: ['Character', 'Floor'], Attributes: [('AttackPower', 10), ('BlocksLight', False), ('BlocksLight', False), ('BlocksMovement', False), ('BlocksMovement', False), ('CanAct', True), ('Health', 100), ('Material', ''), ('MaxHealth', 100)], Positions: (5, 1)\nEntity Types: ['Wall', 'Floor'], Attributes: [('BlocksLight', False), ('BlocksLight', True), ('BlocksMovement', False), ('BlocksMovement', True), ('Material', ''), ('Material', '')], Positions: (3:5, 2), (6:8, 2)\nEntity Types: ['Door', 'Floor'], Attributes: [('BlocksLight', False), ('BlocksLight', True), ('BlocksMovement', False), ('BlocksMovement', True), ('Material', ''), ('Material', ''), ('Open', False), ('is_locked', True), ('required_key', 'Golden Key')], Positions: (5, 2)"

In [ ]:
initial_observation = character_agent.get_current_observation()


In [ ]:
# Debug a single step
print("Initial Observation:")
print(initial_observation)

print("\nSystem Prompt:")
print(system_prompt(
    grid_map.get_actions(),
    agent.use_egocentric,
    agent.use_outlines,
    OutlinesEgoActionPayload,
    OutlinesActionPayload,
    SummarizedEgoActionPayload,
    SummarizedActionPayload
))

print("\nAction Generation Prompt:")
print(action_generation_prompt(
    initial_observation,
    character_agent.goals,
    "",
    agent.use_egocentric,
    agent.use_outlines,
    agent.memory_length,
    OutlinesEgoActionPayload,
    OutlinesActionPayload,
    SummarizedEgoActionPayload,
    SummarizedActionPayload
))



Initial Observation:
Entity Types: ['Floor'], Attributes: [('BlocksLight', False), ('BlocksMovement', False), ('Material', '')], Positions: (0:2, 0:3), (8:10, 0:2), (2, 0:2), (3, 0:2), (4, 0:2), (6, 0:2), (7, 0:2), (5, 0), (9, 2)
Entity Types: ['Character', 'Floor'], Attributes: [('AttackPower', 10), ('BlocksLight', False), ('BlocksLight', False), ('BlocksMovement', False), ('BlocksMovement', False), ('CanAct', True), ('Health', 100), ('Material', ''), ('MaxHealth', 100)], Positions: (5, 1)
Entity Types: ['Wall', 'Floor'], Attributes: [('BlocksLight', False), ('BlocksLight', True), ('BlocksMovement', False), ('BlocksMovement', True), ('Material', ''), ('Material', '')], Positions: (3:5, 2), (6:8, 2)
Entity Types: ['Door', 'Floor'], Attributes: [('BlocksLight', False), ('BlocksLight', True), ('BlocksMovement', False), ('BlocksMovement', True), ('Material', ''), ('Material', ''), ('Open', False), ('is_locked', True), ('required_key', 'Golden Key')], Positions: (5, 2)

System Prompt:
<|im

In [ ]:
print("\nGenerating action...")
action_payload = character_agent.generate_action(initial_observation)
print("Generated Action:")
print(action_payload)

print("\nExecuting action...")
result, error = character_agent.execute_action(action_payload)
if error:
    print("Error:", error)
else:
    print("Action Result:")
    print("State Before:", result["state_before"])
    print("State After:", result["state_after"])


Generating action...



llama_print_timings:        load time =     909.42 ms
llama_print_timings:      sample time =       3.20 ms /    35 runs   (    0.09 ms per token, 10947.76 tokens per second)
llama_print_timings: prompt eval time =    4195.80 ms /  2808 tokens (    1.49 ms per token,   669.24 tokens per second)
llama_print_timings:        eval time =    1077.33 ms /    34 runs   (   31.69 ms per token,    31.56 tokens per second)
llama_print_timings:       total time =    5350.80 ms /  2842 tokens


Generated Action: action_name='MoveStep' target_entity_type='Floor' target_entity_position=[5, 0]
Generated Action:
action_name='MoveStep' target_entity_type='Floor' target_entity_position=[5, 0]

Executing action...
payload detected
converting to summarized
converting summarized payload
conversion results: actions=[ActionInstance(source_id='4176ec77-2d2a-494a-a063-71d0a83f80c1', target_id='75385267-7517-4c4d-9221-ad184aaca349', action=MoveStep(name='Move Step', prerequisites=Prerequisites(source_statements=[Statement(name='Statement', id='99dd2fe6-f6aa-4ace-b467-3eeb98595968', conditions={'can_act': True}, comparisons={}, callables=[])], target_statements=[Statement(name='Statement', id='c04ba669-e56f-4c1f-af79-cf3fc10cfdb9', conditions={'blocks_movement': False}, comparisons={}, callables=[])], source_target_statements=[Statement(name='Statement', id='ae9e672c-ac85-41aa-b6aa-8e24dee76e68', conditions={}, comparisons={'source_position': ('node', 'node', <function source_node_compariso

In [ ]:
print(character.position)

name='Position' id='264317ef-2b4e-4c6f-847c-479347b90117' value=(5, 0)


In [ ]:
# def source_node_at_00(source: Node, target: Optional[Node] = None) -> bool:
#     print("Checking if source node at 0, 0", source.position)
#     if source.node.position.value == (0, 0):
#         print("Source node at 0, 0", source.node.position)
#         return True
#     return False

# # Create the grid map and generate the dungeon
# grid_map = GridMap(width=10, height=10)
# grid_map.register_actions([MoveStep, PickupAction, DropAction, OpenAction, CloseAction, UnlockAction, LockAction])
# room_width, room_height = 6, 6
# character, door, key, treasure = generate_dungeon(grid_map, room_width, room_height)

# # Generate the entity type map
# grid_map.generate_entity_type_map()

# # Create the agent (either FakeLLM or LLMAgent)
# agent = FakeLLM(grid_map, character.id, use_egocentric=True, use_outlines=True)
# # model_path = "./phi-2.Q4_K_M.gguf"  # Replace with the path to your model
# # agent = LLMAgent(grid_map, character.id, model_path)

# # Create the CharacterAgent with the chosen agent
# character_agent = CharacterAgent(grid_map, agent)

# # Set the agent's metadata
# metadata = RunMetadata(character_id=character.id, run_number=1)
# character_agent.set_metadata(metadata)

# # Set the agent's goals
# goal_statement = Statement(callables=[source_node_at_00])  # Example goal: reach position (0, 0)
# goal = AgentGoal(statement=goal_statement, priority=1)
# character_agent.set_goals([goal])

# # Get the initial observation
# initial_observation = character_agent.get_current_observation()
# # initial_observation_ego = character_agent.get_current_observation()



# # Run the agent
# character_agent.run(initial_observation)


In [ ]:
def source_node_at_00(source: Node, target: Optional[Node] = None) -> bool:
    print("Checking if source node at 0, 0", source.position)
    if source.node.position.value == (0, 0):
        print("Source node at 0, 0", source.node.position)
        return True
    return False

# Create the grid map and generate the dungeon
grid_map = GridMap(width=10, height=10)
grid_map.register_actions([MoveStep, PickupAction, DropAction, OpenAction, CloseAction, UnlockAction, LockAction])
room_width, room_height = 6, 6
character, door, key, treasure = generate_dungeon(grid_map, room_width, room_height)

# Generate the entity type map
grid_map.generate_entity_type_map()

# Create the LLMAgent
agent = LLMAgent(grid_map, character.id, model_path, use_egocentric=True, use_outlines=True)

# Create the CharacterAgent with the LLMAgent
character_agent = CharacterAgent(grid_map, agent)

# Set the agent's metadata
metadata = RunMetadata(character_id=character.id, run_number=1)
character_agent.set_metadata(metadata)

# Set the agent's goals
goal_statement = Statement(callables=[source_node_at_00])  # Example goal: reach position (0, 0)
goal = AgentGoal(statement=goal_statement, priority=1)
character_agent.set_goals([goal])

# Get the initial observation
initial_observation = character_agent.get_current_observation()

# Debug a single step
print("Initial Observation:")
print(initial_observation)



Updating block attributes... for door


TypeError: LLMAgent.__init__() got an unexpected keyword argument 'use_outlines'

In [ ]:
print("\nSystem Prompt:")
print(system_prompt(grid_map.get_actions(), agent.use_egocentric, agent.use_outlines))

print("\nAction Generation Prompt:")
print(action_generation_prompt(initial_observation, character_agent.goals, "", agent.use_egocentric, agent.use_outlines, agent.memory_length))



In [ ]:
print("\nGenerating action...")
action_payload = character_agent.generate_action(initial_observation)
print("Generated Action:")
print(action_payload)

print("\nExecuting action...")
result, error = character_agent.execute_action(action_payload)
if error:
    print("Error:", error)
else:
    print("Action Result:")
    print("State Before:", result["state_before"])
    print("State After:", result["state_after"])

In [ ]:
a

In [ ]:
# Debug a single step
print("Initial Observation:")
print(initial_observation)

print("\nSystem Prompt:")
print(agent.system_prompt(registered_actions=grid_map.get_actions()))

print("\nAction Generation Prompt:")
print(agent.action_generation_prompt(initial_observation, character_agent.goals, ""))


In [ ]:

print("\nGenerating action...")
action_payload = character_agent.generate_action(initial_observation)
print("Generated Action:")
print(action_payload)

print("\nExecuting action...")
result, error = character_agent.execute_action(action_payload)
if error:
    print("Error:", error)
else:
    print("Action Result:")
    print("State Before:", result["state_before"])
    print("State After:", result["state_after"])